In [ ]:
import pandas as pd
import json
import nltk
import numpy as np
import re

# 1. Importation des données

In [ ]:
tweet = pd.read_json("datasetProjet2022.json")
seisme = pd.read_csv('Liste_seismes_2017-2022.csv', sep = ';')

In [ ]:
tweet

In [ ]:
seisme

In [ ]:
seisme.isna().sum()
# Aucun champs n'est null pour les seismes

In [ ]:
tweet.columns

In [ ]:
# On ne garde que quelques features qui vont nous interesser pour la suite
tweet=tweet[['hashtags','tweet_text','tweet_created_at']]

In [ ]:
tweet

In [ ]:
# On trie les lignes de tweet par date croissante
tweet=tweet.sort_values('tweet_created_at').reset_index()
tweet=tweet.drop('index',axis=1)

In [ ]:
# on défini la date comme le nouvel index
tweet=tweet.set_index(tweet['tweet_created_at'])
tweet=tweet.drop('tweet_created_at',axis=1)

In [ ]:
tweet

In [ ]:
#Prélèvement des dates et heure des seismes
seisme=seisme['Date Heure']

In [ ]:
seisme

In [ ]:
#Arrondissement des dates de séisme à la seconde près
seisme=seisme.map(lambda x: x[:-3])

In [ ]:
# Conversion de la date en datetime
seisme=pd.to_datetime(seisme)

In [ ]:
# Test du format datetime
seisme[0]-pd.Timedelta('5h')


Les opérations sur les dates fonctionnement, c'est validé !!!

# 2. Etiquetage des tweets

Idéalement, il faudrait des tweets labellisés à la main. Ce n'est pas le cas ici !!

# 2.1 Les fenetres de séismes

Pour labelliser les tweets, on fera l'hypothèse qu'un séisme a un impact sur tweeter pendant les 12h qui suivent son apparition. Ce pas de temps mériterait d'être étudier pour trouver la valeur qui optimise les prédictions de notre algorithme de ML.

In [ ]:
# Créer la fonction construisant les fenetres de tremblement de terre

def get_window(seisme,window_hour):
    windows=[]
    string=str(window_hour)+'h'
    for elem in seisme:
        windows.append((elem,elem+pd.Timedelta(string)))
    return windows

# Hypothèse : si un séisme survient, les 12 heures suivantes font partie de la fenetre temporelle de ce tremblement de terre
windows=get_window(seisme,12)

In [ ]:
windows

# 2.2 Etiquetage binaire des tweets : 

On utilise les fenetres de tremblement de terre ci-dessus

Labéllisés à 1 s'ils appartiennent à une fenetre de tremblement de terre, 0 sinon

In [ ]:
# Initialisation de la feature seism_association
tweet['seism_association']=np.zeros(len(tweet))

In [ ]:
# Les tweets sont labéllisés
for elem in tweet.index:
    for elem2 in windows:
        if elem > elem2[0] and elem < elem2[1]:
            tweet.loc[elem,'seism_association']=1  # Si le tweet est dans une fenetre de tremblement de terre, il est positif
            pass
        

In [ ]:
# Nombre de tweets étiquetés à 1
tweet['seism_association'].sum()

31 423 tweets sont positifs. Cela représente une faible part des 500 000 tweets. Il faut rééquilibrer le dataset !

In [ ]:
# Visualiser les tweets positifs
tweet[tweet['seism_association']==1]

Certains tweets sont labellisés positifs alors qu'ils ne parlent pas d'un vrai séisme. On devra le prendre en compte dans l'étude des résultats

In [ ]:
# Exemple de tremblement de terre mal labellisé
tweet_positif = tweet[tweet['seism_association']==1]
tweet_positif.iloc[13].tweet_text

# 3. Rééquilibrer le dataset de tweet

In [ ]:
# Supprimer une bonne part des tweets négatifs
list_suppr=[]  # liste de tweets à supprimer
i=0
while i<len(tweet):
    if tweet.seism_association[i]==0:  # si le tweet est labelisé "negatif"
        alea = random.random()
        if alea>=0.20: # On jette au hazard 80% des tweets
            list_suppr.append(tweet.index[i])
    i=i+1        
tweet.drop(list_suppr,0,inplace=True)
len(tweet)

Il reste environ 125 000 tweets dont 32 000 labéllisés positifs. Jetter de façon aléatoire des tweets pourrait causer des problèmes de fréquences de tweets dans notre modèle de ML. On garde cela en tête.

# 4. Nettoyer le texte

In [ ]:
# Remplacer les caractères avec accents par les lettres correspondantes
import re
from datetime import datetime
from thefuzz import fuzz


def de_accentize(text):
    """
    Remove usual latin language accentuation from letters (uppercase as well as lowercase)
    """
    accentedChars =    'àÀãÃéÉèÈëËíÍîÎóÓõÕôÔúÚûÛùÙñÑÇç'
    de_accentedChars = 'aaaaeeeeeeiiiioooooouuuuuunncc'
    transTable = str.maketrans(accentedChars,de_accentedChars)
    return text.translate(transTable)

In [ ]:
# Nettoyer la feature texte (tout en minuscule, pas de caractère spécial,...)
texte_list=[]
for i in range(len(tweet)):
    texte = tweet['tweet_text'][i]
    texte=de_accentize(texte)
    texte = texte.lower()
    texte = re.sub('((www\.[\s]+)|(https?://[^s]+))','', texte)
    texte = re.sub("@[A-Za-z0-9_]+","", texte)
    texte = re.sub("#[A-Za-z0-9_]+","", texte)
    texte = re.sub('[()!?]', ' ', texte)
    texte = re.sub('\[.*?\]',' ', texte)
    texte = re.sub("[^a-z0-9]"," ", texte)
    texte_list.append(texte)
tweet['texte_nettoye'] = texte_list
tweet['texte_nettoye']

# 5. Créer de nouvelles features à partir du texte

# 5.1 Feature nombre de mots par tweet

In [ ]:
# Création de la feature nombre de mot par tweet
nb_mot_list=[]
for phrase in tweet['texte_nettoye']:
    nb_mot_list.append(len(phrase.split()))
# Mettre à jour la feature avec le nombre de mots
tweet['nb_mot']=nb_mot_list

# 5.2 Features mots relatifs au champs lexical du séisme

In [ ]:
# Fonction concaténant le texte d'un vecteur
def unpack(L):
    unpacked=''
    for i in range (len(L)):
        unpacked+=L[i]
    return unpacked
        

In [ ]:
# Concaténation du texte des tweets positifs
unpack(text_valid)

In [ ]:
# Création du dictionnaire de tokens et fréquence
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from nltk.corpus import stopwords

tokenizer=TweetTokenizer()
tokens=tokenizer.tokenize(unpack(text_valid))
freq=nltk.FreqDist(tokens)
for w in sorted(freq, key=freq.get, reverse=True):
  print (w, freq[w])

Dans ce dictionnaire de tokens, on peut retrouver des mots du champs lexical du séisme tel que "seisme", "magnitude", "tremblements" et "terre" avec des occurences importantes. On peut donc créer des features pour chacun de ces mots clés

In [ ]:
# Features du champs léxical du séisme
mot_seisme=np.zeros(len(tweet))
mot_tremblement=np.zeros(len(tweet))
mot_terre=np.zeros(len(tweet))
mot_magnitude=np.zeros(len(tweet))
for i in range(len(tweet)):
    tok=tokenizer.tokenize(tweet.texte_nettoye[i])
    if "seisme" in tok or "seismes" in tok:
        mot_seisme[i]=1
    if "tremblement" in tok or "tremblements" in tok:
        mot_tremblement[i]=1
    if "terre" in tok:
        mot_terre[i]=1
    if "magnitude" in tok:
        mot_magnitude[i]=1
tweet['mot_seisme']=mot_seisme
tweet['mot_tremblement']=mot_tremblement
tweet['mot_terre']=mot_terre
tweet['mot_magnitude']=mot_magnitude
tweet

In [ ]:
# Affichage du texte du 11eme tweet du dataframe
tweet.iloc[11].tweet_text

In [ ]:
# Affichage du 11eme tweet
tweet.iloc[11]

On observe que le texte du tweet et les features "mot_..." sont en adéquation.

# 5.3 Feature sentiment

In [ ]:
# Test de l'analyse de sentiment
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
texte_nettoye = "C'est incroyable j'ai réussi à relever le plus gros défi de ma vie je suis tellement heureux"
sentiment = TextBlob(texte_nettoye,pos_tagger=PatternTagger(),analyzer=PatternAnalyzer()).sentiment[0]
sentiment

In [ ]:
# Initialisation de la feature sentiment
tweet['sentiment']=np.zeros(len(tweet))

In [ ]:
#Créer la feature sentiment

from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

emotion = np.zeros(len(tweet))
for i in range(len(tweet)):
    emotion[i] = TextBlob(tweet.texte_nettoye[i],pos_tagger=PatternTagger(),analyzer=PatternAnalyzer()).sentiment[0]
tweet.sentiment = emotion

# 5.4 Feature fréquence

Attention : La cellule ci dessous met 6H à tourner (sur mon PC...)

En effet cette feature a une complexité en 2n², il y a environ 125 000 tweets ce qui implique un temps de calcul très long

In [ ]:
#La fréquence locale d'un tweet, c'est le nombre de tweets publiés dans un "rayon" de 12h autour de la date du tweet

# Initialiser la feature frequence
tweet['frequence']=np.zeros(len(tweet))
# initialiser la liste des fréquences
freq_list=np.zeros(len(tweet))
delta= '12h'  
for i in range(len(tweet)):
    time_tweet=tweet.index[i] #Date de publication du tweet
    debut_window=time_tweet-pd.Timedelta(delta) # Date de début de la fenetre locale du tweet
    fin_window=time_tweet+pd.Timedelta(delta) # Date de fin de la fenetre locale du tweet
    df1=tweet[tweet.index>=debut_window] # Prélèvement des tweets dont la date est supérieure à la date de début
    df2=tweet[tweet.index<fin_window]  # Prélèvement des tweets dont la date est infèrieure à la date de fin
    df3=pd.merge(df1,df2) # Intersection des 2 tableaux ci-dessus
    freq=len(df3) # frequence locale calculée pour le tweet en question
    freq_list[i]=freq
    print("i="+str(i)) # Cet affichage permet de vérifier que le programme tourne toujours après quelques heures....
    print(freq)
    
tweet['frequence']=freq_list
tweet['frequence']

In [ ]:
# Affichage de la frequence locale d'un tweet
tweet['frequence'][50]

In [ ]:
# Courbe de frequence locale en fonction du temps
import matplotlib.pyplot as plt
plt.plot(tweet.index, tweet.frequence)
plt.ylabel('Fréquence des tweets en fonction du temps')
plt.show()

Globalement, on observe une étrange gaussienne pour la répartition des tweets au cours des années. On peut supposer que le covid a entrainé une augmentation des tweets. Les différents pics peuvent être du à la suppression aléatoire des tweets négatifs opérée plus haut ou alors à l'apparition d'un tremblement de terre.

In [ ]:

plt.plot(tweet.index[47000:72000], tweet.frequence[47000:72000])
plt.ylabel('Fréquence des tweets en fonction du temps')
plt.show()

En regardant la liste des séismes, on retrouve bien un séisme en fin juin 2019 et en novembre 2019 ce qui correspond à nos pics de tweets sur la courbe.

Il serait interessant de définir une nouvelle feature : la variation de la fréquence qui permettrait de mettre en valeur les augmentations soudaines 

# 5.5 Feature écart entre fréquence locale et la fréquence au loin

Idéalement, on devrait calculer un taux d'accroissement pour la frequence pour approximer la dérivée.

Faute de temps, on fera une approximation grossière en ne s'interessant qu'à l'écart entre la fréquence locale et la fréquence au loin.

hypothèse : la fréquence au loin pour le tweet i, c'est la moyenne entre la fréquence de tweet[i-pas] et tweet[i+pas]. Après plusieurs tests, pas=550 semble etre le parametre optimal pour cette feature

In [ ]:
variation_freq=np.zeros(len(tweet))
pas = 550
for i in range(len(tweet)):
    if i<pas:
        moy_freq = (tweet.frequence[i-pas]+tweet.frequence[i+pas])/2
    else:
        moy_freq = (tweet.frequence[i-pas]+tweet.frequence[i+pas-len(tweet)])/2
    variation_freq[i]=abs(tweet.frequence[i]-moy_freq)
tweet['variation_freq']=derive_freq

In [ ]:
# Courbe de frequence locale en fonction du temps
import matplotlib.pyplot as plt
plt.plot(tweet.index, tweet.variation_freq)
plt.ylabel('Variation de la frequence des tweets en fonction du temps')
plt.show()

# 5.6 Feature nombre d émojis par tweet

In [ ]:
# Test pour compter les emojis
import advertools as adv
text_list = ['I feel like playing basketball 🏀',
             'I like playing football ⚽⚽',
             'Not feeling like sports today']

emoji_summary = adv.extract_emoji(text_list)
print(emoji_summary)

In [ ]:
# Création de la feature représentant le nombre d'émoji de chaque tweet
nb_emoj=np.zeros(len(tweet))
tweet['nb_emoji']=np.zeros(len(tweet))
for i in range(len(tweet)):
    emoji_summary = adv.extract_emoji(tweet.tweet_text[i])
    nb_emoj[i] = emoji_summary['overview']['num_emoji']
    print(i)
tweet['nb_emoji'] = nb_emoj

In [ ]:
tweet[tweet.mot_magnitude==1]

# 6 Visualisation

In [ ]:
# Visualisation de chaque variable sous forme d'un histogramme
import matplotlib.pyplot as plt

tweet_clean=tweet[['nb_mot', 'frequence', 'nb_emoji', 'variation_freq', 'sentiment','mot_seisme','mot_tremblement', 'mot_terre', 'mot_magnitude', 'seism_association']]
tweet_clean.hist(bins=50,figsize=(20,15))
plt.show()

La feature sentiment reste majoritairement aux alentours de 0 avec beaucoup de valeures nulles

L'équilibre relatif entre les 1 et les 0 des features de mots est réjouissant.

On peut voir que le nombre de tweets positifs représente 1/5 des tweets via la feature seism_association, et on se félicite d'avoir tenté d'équilibrer le jeu de donnés.

On peut voir également que le nombre de mots suit une distribution de gauss.

La fréquence est répartie de façon homogène tandis que la variation de la fréquence est une demi gaussienne (du à la valeur absolue).

Les données nous conviennent on peut maintenant passer à l'étude des corrélations.

In [ ]:
# Calcul de la matrice des coefficients de Pearson 
corr_matrix=tweet_clean.corr()

In [ ]:
# Visualisation de la matrice des coefficients de Pearson
import seaborn as sns
heat_map = sns.heatmap(corr_matrix, center=0, annot=True)

La fréquence est la feature qui présente le plus de corélation avec les labels (seism_association) avec corr=0,5. Ca sera donc une feature très interressante pour prédire les tremblements de terre et on pouvait s'y attendre.

Les features concernant le champs lexical du séisme sont corrélés entre elles et ça aussi c'est cohérent. La présence des mots "tremblements" et "terre" est fortement corrélé aux labels. Pour les mots magnitude et séisme, on retrouve peu de corrélation ce qui s'explique par le fait que sur tweeter, le langage soutenu est moins utilisé. D'un autre coté, les tweets étant mal étiquetés, on pourrait s'attendre à voir les scores de corrélation augmenter pour beaucoup des features ci dessus.

# 7 Normalisation des données

In [ ]:
# Initialisation des features à normaliser
X = tweet_clean.drop('seism_association', axis=1)
y = tweet_clean['seism_association']

In [ ]:
# Normalisation des features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()  # normalisation
scaler.fit(X)
X_stand = scaler.transform(X)

# 8. METHODE RANDOMFOREST

Initialisation :

In [ ]:
# Initialisation des entrées du modèle
X_train, X_test, y_train, y_test = X_stand[:100000], X_stand[100001:], y[:100000] , y[100001:]
y_test.sum()

Entrainement :

In [ ]:
# Entrainement du modèle
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators= 40) # choix de l'hyper paramètre : 40 = Nombre d'arbres de la foret
clf.fit(X_train, y_train) # entrainement

Prédiction :

In [ ]:
# Prédiction sur les X_test à partir de notre modèle entrainé
y_predict=clf.predict(X_test)

In [ ]:
# Affichage du nombre de tweets correspondant à un tremblement de terre selon notre modèle
y_predict.sum()

La prédiction donne 1 259 tweets positifs contre 3 977 pour les y_test

Evaluation :

In [ ]:
# Evaluation des métriques pour notre modèle entrainé
from sklearn.metrics import precision_score, recall_score
print(precision_score(y_test, y_predict))
print(recall_score(y_test, y_predict))

25% de précision c'est clairement un score pourris. On peut supposer que si les tweets étaient labéllisés à la main le résultat serait meilleur.

In [ ]:
y_test[2000]

In [ ]:
y_predict[2000]

On observe que le tweet 2 000 a été mal classifié par notre modèle. C'est tout à fait normal vu la faible précision de notre modèle.

# Pour synthétiser

On a réussi a labelliser les tweets de façon grossière mais les résultats ne seront pas bon tant que les tweets ne seront pas labellisés de façon plus precise. On pourrait imaginer classifier à la main les 32 000 tweets labellisés positifs par exemple. Sinon, labélliser 32 000 tweets ça coute environ 1 300€ donc c'est abordable.

La labellisation des tweets fait apparaitre un paramètre : le temps d'impact d'un séisme sur tweeter. Il vaut 12h dans ce modèle mais cette valeure n'est pas forcément celle optimale.

Un rééquilibrage du dataset a permit de travailler avec un dataset présentant 20% de tweets positifs.

La feature fréquence fait apparaitre un autre paramètre : la durée autour du tweet pour le calcul de sa fréquence. Elle vaut 12h également et ce paramètre peut être optimisé. Malheureusement, cette feature a un temps de calcul très long mais doit pouvoir s'optimiser facilement. Nous n'avons pas réussi à diminuer le temps de calcul en dessous de 6h de notre coté.

La feature variation de fréquence est calculée par une approximation grossière qui ferait peur à un matématicien. Cependant elle nous a permis, une fois implémenté, de passer de 7% d'accuracy à 30% pour certains entrainements de randomforest. Elle fait apparaitre un nouveau paramètre : le pas de calcul pour la variation de fréquence. Il vaut 550 actuellement. Il serait encore plus interessant de remplacer cette feature par un taux d'accroissement. 

Les features les plus interressantes sont la fréquence, la variation de la fréquence, la présence de mots du champs léxical du séisme. On pourrait étudier d'avantage le texte pour en extraire d'autres features interessantes. Cependant, on remarque que les features telles que le nombre de mots, le nombre d'émojis ou l'analyse de sentiment n'apporte pas vraiment de résultat concluant. En effet, la correlation entre ces features et la présence d'un séisme est très proche de 0.

Notre modèle ne nous permet pas de classifier les tweets de façon concluante et nous obtenons au mieux 30% de prédictions justes mais on peut espérer qu'avec une labéllisation plus juste on pourrait obtenir de meilleures prédictions.

La piste de recherche la plus prommeteuse serait de passer en Deep Learning en utilisant un réseau de neurones récurent tel que un réseau LSTM : Chaque tweet possède un texte. Une fois nettoyé, ce texte est transformé en un vecteur via de l'embedding. On a donc une séquence de mots transformés en une séquence de nombres. Ainsi, on peut fournir en entrée du réseau LSTM les vecteurs qui sont des séquences correspondant au texte des tweets. Au bout du réseau LSTM, on met une couche de neuronnes dense permettant de classifier de facon binaire le tweets. 

Cette solution permettrait de traduire tout le texte de chaque tweet en tenant compte de l'ordre des mots des phrases qui est très important pour leur compréhension.

